## Gold Layer Creation

In [1]:
from pyspark.sql.functions import *

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 3, Finished, Available, Finished)

### Date Dimension

In [3]:
customers_table = spark.read.table("fabricPulse_lakehouse.silver.customers").alias("ct")
customers_table.show(5)

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 5, Finished, Available, Finished)

+-----------+--------------------+----------+---------+--------------------+---------------+--------------------+
|customer_id|               email|first_name|last_name|             country|customer_status|          created_at|
+-----------+--------------------+----------+---------+--------------------+---------------+--------------------+
|   bdd640fb|  john21@example.net|       Joy|  Gardner|               Nauru|       VERIFIED|2024-11-08 16:50:...|
|   3b8faa18|hoffmanjennifer@e...|       Amy| Robinson|             Namibia|         BANNED|2024-07-26 17:00:...|
|   ce4a2bbd|helenpeterson@exa...|     Jason|Gallagher|Bosnia and Herzeg...|       VERIFIED|2024-03-13 05:52:...|
|   580d7b71|lindsay78@example...|    Teresa|     Gray|          Micronesia|       VERIFIED|2024-08-06 15:45:...|
|   dc98d2c1|barbara10@example...|    Jeremy|  Roberts|             Belgium|       VERIFIED|2025-09-21 06:55:...|
+-----------+--------------------+----------+---------+--------------------+------------

In [5]:
dim_customers = customers_table.withColumn("customer_key", monotonically_increasing_id()) \
.select("customer_key","ct.*")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 7, Finished, Available, Finished)

In [7]:
dim_customers.show(5)

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 9, Finished, Available, Finished)

+------------+-----------+--------------------+----------+---------+--------------------+---------------+--------------------+
|customer_key|customer_id|               email|first_name|last_name|             country|customer_status|          created_at|
+------------+-----------+--------------------+----------+---------+--------------------+---------------+--------------------+
|           0|   bdd640fb|  john21@example.net|       Joy|  Gardner|               Nauru|       VERIFIED|2024-11-08 16:50:...|
|           1|   3b8faa18|hoffmanjennifer@e...|       Amy| Robinson|             Namibia|         BANNED|2024-07-26 17:00:...|
|           2|   ce4a2bbd|helenpeterson@exa...|     Jason|Gallagher|Bosnia and Herzeg...|       VERIFIED|2024-03-13 05:52:...|
|           3|   580d7b71|lindsay78@example...|    Teresa|     Gray|          Micronesia|       VERIFIED|2024-08-06 15:45:...|
|           4|   dc98d2c1|barbara10@example...|    Jeremy|  Roberts|             Belgium|       VERIFIED|2025-0

### Accounts Dimension

In [9]:
accounts_table = spark.read.table("fabricPulse_lakehouse.silver.accounts").alias("at")
accounts_table.show(5)

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 11, Finished, Available, Finished)

+-----------+----------+------------+--------+--------------+--------------------+--------------+
|customer_id|account_id|account_type|currency|account_status|           opened_at|valid_customer|
+-----------+----------+------------+--------+--------------+--------------------+--------------+
|   4cac895c|ACC-182155|     SAVINGS|     EUR|        ACTIVE|2024-03-30 18:34:...|          true|
|   dc432ed5|ACC-422700|     SAVINGS|     GBP|        ACTIVE|2025-05-21 21:57:...|          true|
|   eb9e54e0|ACC-948092|    CHECKING|     USD|        ACTIVE|2025-05-30 15:11:...|          true|
|   d5f25073|ACC-333578|    CHECKING|     USD|        ACTIVE|2024-06-17 18:41:...|          true|
|   702cdd20|ACC-542404|     SAVINGS|     EUR|        ACTIVE|2024-03-23 03:55:...|          true|
+-----------+----------+------------+--------+--------------+--------------------+--------------+
only showing top 5 rows



In [10]:
dim_accounts = accounts_table.withColumn("account_key",monotonically_increasing_id()) \
.join(dim_customers,on="customer_id",how="left").select("account_key","account_id","customer_key","account_type","currency","account_status","opened_at","valid_customer")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 12, Finished, Available, Finished)

In [11]:
dim_accounts.show()

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 13, Finished, Available, Finished)

+-----------+----------+------------+------------+--------+--------------+--------------------+--------------+
|account_key|account_id|customer_key|account_type|currency|account_status|           opened_at|valid_customer|
+-----------+----------+------------+------------+--------+--------------+--------------------+--------------+
|          0|ACC-182155|         962|     SAVINGS|     EUR|        ACTIVE|2024-03-30 18:34:...|          true|
|          1|ACC-422700|         454|     SAVINGS|     GBP|        ACTIVE|2025-05-21 21:57:...|          true|
|          2|ACC-948092|         535|    CHECKING|     USD|        ACTIVE|2025-05-30 15:11:...|          true|
|          3|ACC-333578|          52|    CHECKING|     USD|        ACTIVE|2024-06-17 18:41:...|          true|
|          4|ACC-542404|          29|     SAVINGS|     EUR|        ACTIVE|2024-03-23 03:55:...|          true|
|          5|ACC-166140|         587|    CHECKING|     EUR|        ACTIVE|2024-04-11 09:22:...|          true|
|

### Instruments Dimension

In [12]:
instruments_table = spark.read.table("fabricPulse_lakehouse.silver.instruments")
instruments_table.show(5)

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 14, Finished, Available, Finished)

+-------------+-----------+--------+--------+--------+-------+
|instrument_id|       name|category|provider|currency| status|
+-------------+-----------+--------+--------+--------+-------+
|     INST-001| Apple Inc.|  EQUITY|  NASDAQ|     USD|TRADING|
|     INST-002| Tesla Inc.|  EQUITY|  NASDAQ|     USD|TRADING|
|     INST-003|    Bitcoin|  CRYPTO| BINANCE|     BTC|TRADING|
|     INST-004|   Ethereum|  CRYPTO| BINANCE|     ETH|TRADING|
|     INST-005|S&P 500 ETF|   INDEX|VANGUARD|     USD|TRADING|
+-------------+-----------+--------+--------+--------+-------+
only showing top 5 rows



In [15]:
dim_instruments = instruments_table.withColumn("instrument_key",monotonically_increasing_id()) \
.select("instrument_key","instrument_id","name","category","provider","currency","status")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 17, Finished, Available, Finished)

In [16]:
dim_instruments.show(5)

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 18, Finished, Available, Finished)

+--------------+-------------+-----------+--------+--------+--------+-------+
|instrument_key|instrument_id|       name|category|provider|currency| status|
+--------------+-------------+-----------+--------+--------+--------+-------+
|             0|     INST-001| Apple Inc.|  EQUITY|  NASDAQ|     USD|TRADING|
|             1|     INST-002| Tesla Inc.|  EQUITY|  NASDAQ|     USD|TRADING|
|             2|     INST-003|    Bitcoin|  CRYPTO| BINANCE|     BTC|TRADING|
|             3|     INST-004|   Ethereum|  CRYPTO| BINANCE|     ETH|TRADING|
|             4|     INST-005|S&P 500 ETF|   INDEX|VANGUARD|     USD|TRADING|
+--------------+-------------+-----------+--------+--------+--------+-------+
only showing top 5 rows



### Transactions Fact

In [19]:
transactions_table = spark.read.table("fabricPulse_lakehouse.silver.transactions").alias("tt")
transactions_table.show(5)

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 21, Finished, Available, Finished)

+----------+--------------+-------+---------+----------------+------------------+--------+-------------+--------------------+---------------------+-------------+
|account_id|transaction_id| amount|direction|transaction_type|transaction_status|currency|instrument_id|           timestamp|parent_transaction_id|valid_account|
+----------+--------------+-------+---------+----------------+------------------+--------+-------------+--------------------+---------------------+-------------+
|ACC-327406|  5c1e2275-c09|6396.76| OUTBOUND|      WITHDRAWAL|         COMPLETED|     EUR|     INST-002|2025-10-02 22:50:...|                 NULL|         true|
|ACC-940983|  49cc9719-9cc|8917.58|  INBOUND|           TRADE|         COMPLETED|     GBP|     INST-004|2025-04-21 00:09:...|                 NULL|         true|
|ACC-940983|  FEE-49cc9719|  17.84| OUTBOUND|             FEE|         COMPLETED|     GBP|INST-CASH-USD|2025-04-21 00:09:...|         49cc9719-9cc|         true|
|ACC-669396|  f8f7c535-e81|7

In [28]:
dim_date = transactions_table.select(col("timestamp").cast("Date").alias("date"))

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 30, Finished, Available, Finished)

In [40]:
dim_date = dim_date.withColumns({
    "date_key":date_format("date","ddMMyyyy").cast("int"),
    "day":day("date"),"day_of_week":date_format("date","EEEE"),"month":month("date"),"quarter":quarter("date"),"year":year("date"),
    "is_weekend": when((date_format("date","EEEE") == "Saturday") | (date_format("date","EEEE") == "Sunday"),True).otherwise(False)
    }).select("date_key","date","day","day_of_week","month","quarter","year","is_weekend")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 42, Finished, Available, Finished)

In [42]:
fact_transactions = transactions_table.withColumn("transaction_key",monotonically_increasing_id()).withColumn("date",col("timestamp").cast("date").alias("date")) \
    .join(dim_instruments,on="instrument_id",how="left") \
    .join(dim_accounts,on="account_id",how="left") \
    .join(dim_customers,on="customer_key",how="left") \
    .join(dim_date,on="date",how="left") \
    .select("transaction_key","transaction_id","date_key","account_key","instrument_key","customer_key","amount","direction","transaction_type","transaction_status","tt.currency","timestamp","parent_transaction_id","valid_account")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 44, Finished, Available, Finished)

In [43]:
fact_transactions = fact_transactions.filter("valid_account = true AND transaction_status = 'COMPLETED'")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 45, Finished, Available, Finished)

## Creating Gold Layer Tables

In [46]:
dim_accounts.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.gold.dim_accounts")
dim_customers.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.gold.dim_customers")
dim_date.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.gold.dim_date")
dim_instruments.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.gold.dim_instruments")
fact_transactions.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.gold.fact_transactions")

StatementMeta(, ef519b71-5126-49ae-9255-f24b9bc5b990, 48, Finished, Available, Finished)